# Assignment #3 - Storage and Filesystems

I have attached 5 virtual disks to your VM. Each disk is 1 GiB.
You will be using all 5 virtual disks for this assignment. 

Be very careful to not make any changes to your primary disk `/dev/sda`

## Part 1 - Partitioning (40 points)

For this first part, you're going to do the following:

- View information about the virtual disks
- Partition a virtual disk
- Create a filesystem on the disk
- Mount the disk
- Create a large test file on the mounted disk
- Unmount the disk
- Shrink the filesystem while keeping the large test file intact
- Shrink the partition
- Expand the filesystem to match the new partition size
- Remount the disk
- Wipe the disk to start fresh for part 2

In [1]:
# List all disks
lsblk

NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINTS
loop0    7:0    0  73.9M  1 loop /snap/core22/1748
loop1    7:1    0 104.2M  1 loop /snap/core/17200
loop2    7:2    0 104.8M  1 loop /snap/lxd/31214
loop3    7:3    0 112.9M  1 loop /snap/lxd/32455
loop4    7:4    0  44.4M  1 loop /snap/snapd/23545
sda      8:0    0    42G  0 disk 
├─sda1   8:1    0   512M  0 part /boot/efi
├─sda2   8:2    0  40.5G  0 part /
└─sda3   8:3    0   976M  0 part [SWAP]
sdb      8:16   0     1G  0 disk 
sdc      8:32   0     1G  0 disk 
sdd      8:48   0     1G  0 disk 
sde      8:64   0     1G  0 disk 
sdf      8:80   0     1G  0 disk 


In [2]:
# In this cell, use lsblk with the -d flag to only print the five 1G disks
lsblk -d -o NAME,SIZE,TYPE | grep ' 1G'


sdb       1G disk
sdc       1G disk
sdd       1G disk
sde       1G disk
sdf       1G disk


### Now that you have the `/dev` addresses of all five 1G disks, you're going to partition them.

In [3]:
# Install parted
sudo apt -y install parted

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libparted2
Suggested packages:
  libparted-dev libparted-i18n parted-doc
The following NEW packages will be installed:
  libparted2 parted
0 upgraded, 2 newly installed, 0 to remove and 3 not upgraded.
Need to get 333 kB of archives.
After this operation, 662 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bookworm/main amd64 libparted2 amd64 3.5-3 [294 kB]
Get:2 http://deb.debian.org/debian bookworm/main amd64 parted amd64 3.5-3 [39.1 kB]
Fetched 333 kB in 0s (2,313 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline

78Selecting previously unselected package libparted2:amd64.
(Reading database ... 97622 files and directories currently installe

### In a terminal, use `parted` to create a GPT partition table on `/dev/sdb`

In [5]:
# Verify that you created a gpt partition table on /dev/sdb
sudo parted /dev/sdb print | grep 'Partition Table:'

Partition Table: gpt


### In a terminal, use `parted` to create a single partition that spans the entirety of `/dev/sdb`

In [6]:
# Verify that you created the partition correctly
# You should see a single partition with a size of 1072MB
sudo parted /dev/sdb print

Model: QEMU QEMU HARDDISK (scsi)
Disk /dev/sdb: 1074MB
Sector size (logical/physical): 512B/512B
Partition Table: gpt
Disk Flags: 

Number  Start   End     Size    File system  Name     Flags
 1      1049kB  1073MB  1072MB               primary



In [7]:
# Show the partition with lsblk
lsblk | grep sdb

sdb      8:16   0     1G  0 disk 
└─sdb1   8:17   0  1022M  0 part 


### Format the partition as ext4

In [8]:
# In this cell, write the command to format /dev/sdb1 as ext4
sudo mkfs.ext4 /dev/sdb1

mke2fs 1.47.0 (5-Feb-2023)
Discarding device blocks: done                            
Creating filesystem with 261632 4k blocks and 65408 inodes
Filesystem UUID: 504f0ce2-4f13-4e01-84f4-429214dbb010
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (4096 blocks): done
Writing superblocks and filesystem accounting information: done



In [9]:
# Verify that the filesystem was created correctly
sudo blkid /dev/sdb1

/dev/sdb1: UUID="504f0ce2-4f13-4e01-84f4-429214dbb010" BLOCK_SIZE="4096" TYPE="ext4" PARTLABEL="primary" PARTUUID="405eb66d-0651-464a-8111-b1847aeb2515"


### Create a directory at `/mnt/disk1` and mount `/dev/sdb1` to it using the `mount` command

In [10]:
# In this cell, write the command to create the /mnt/disk1 directory
sudo mkdir /mnt/disk1

In [11]:
# In this cell, write the command to mount /dev/sdb1 to /mnt/disk1
sudo mount /dev/sdb1 /mnt/disk1

In [12]:
# Verify that it's mounted correctly
mount | grep /mnt/disk1

/dev/sdb1 on /mnt/disk1 type ext4 (rw,relatime)


## Part 2 - Safely shrinking partitions

In [13]:
# Create a 100 MB file in /mnt/disk1
sudo dd if=/dev/zero of=/mnt/disk1/100MB_file bs=1M count=100

100+0 records in
100+0 records out
104857600 bytes (105 MB, 100 MiB) copied, 0.133424 s, 786 MB/s


In [14]:
# Verify the file was created successfully
ls -lh /mnt/disk1/100MB_file

-rw-r--r-- 1 root root 100M Apr 16 08:18 /mnt/disk1/100MB_file


In [15]:
# In this cell, write the command to unmount /mnt/disk1 using the umount command
# HINT: Make sure your terminal is not currently inside /mnt/disk1 or else you will get a 'target busy' error
sudo umount /mnt/disk1

In [16]:
# Verify filesystem integrity before shrinking
sudo e2fsck -f /dev/sdb1

e2fsck 1.47.0 (5-Feb-2023)
Pass 1: Checking inodes, blocks, and sizes
Pass 2: Checking directory structure
Pass 3: Checking directory connectivity
Pass 4: Checking reference counts
Pass 5: Checking group summary information
/dev/sdb1: 12/65408 files (8.3% non-contiguous), 34451/261632 blocks


### Shrink the `/dev/sdb1` filesystem to its minimum size (approximately 150MB due to the large file you created)

In [17]:
# In this cell, write a command using `resize2fs` to shrink /dev/sdb1 to its minimum size
# HINT: There is a flag that automatically shrinks to the minimum size without you having to calculate it
sudo resize2fs -M /dev/sdb1

resize2fs 1.47.0 (5-Feb-2023)
Resizing the filesystem on /dev/sdb1 to 37655 (4k) blocks.
The filesystem on /dev/sdb1 is now 37655 (4k) blocks long.



### In a terminal, use `parted` to shrink the `/dev/sdb1` partition to 250MB

In [18]:
# Verify that the partition was shrunk to 250MB
sudo parted /dev/sdb print

Model: QEMU QEMU HARDDISK (scsi)
Disk /dev/sdb: 1074MB
Sector size (logical/physical): 512B/512B
Partition Table: gpt
Disk Flags: 

Number  Start   End    Size   File system  Name     Flags
 1      1049kB  250MB  249MB  ext4         primary



In [19]:
# In this cell, write the command to mount /dev/sdb1 to /mnt/disk1
# Hint: You already did this once, so just copy the same command from before
sudo mount /dev/sdb1 /mnt/disk1


In [20]:
# Verify that the drive is mounted again
mount | grep /mnt/disk1

/dev/sdb1 on /mnt/disk1 type ext4 (rw,relatime)


### Use `resize2fs` to expand the filesystem to match the 250 MB partition size

In [21]:
# In this cell, write the command to resize the filesystem to fully occupy the 250 MB partition
sudo resize2fs /dev/sdb1


resize2fs 1.47.0 (5-Feb-2023)
Filesystem at /dev/sdb1 is mounted on /mnt/disk1; on-line resizing required
old_desc_blocks = 1, new_desc_blocks = 1
The filesystem on /dev/sdb1 is now 60779 (4k) blocks long.



In [22]:
# Verify that the filesystem is now expanded to fit the partition
# It will not be exactly 250 MB, but it should be over 200 MB
df -h /mnt/disk1

Filesystem      Size  Used Avail Use% Mounted on
/dev/sdb1       217M  101M  107M  49% /mnt/disk1


# Short answer (10 points)

You shrunk the filesystem before the partition because shrinking the partition first can be destructive. Filesystems must always be contained fully within a partition to avoid filesystem corruption and data loss. In Part 1, you shrunk a filesystem to a smaller size (~150 MB) than you shrunk the partition to (250MB), then you expanded the filesystem to occupy the new 250MB partition size. While this is not necessary, why might this be safer than simply shrinking the filesystem to 250MB and then shrinking the partition to the exact same size?

Write your answer in the cell below.

Shrinking the filesystem before shrinking the partition is much safer because it verifies that the filesystem itself is smaller than the partition before making any changes to the partition size. If you shrink the partition first, there's a risk that the filesystem might be bigger than the partition’s new limits, which could result in filesystem corruption or data loss. By shrinking the filesystem first, you make sure it fits within the partition, reducing the risk of data being lost when the partition size is reduced.

# Part 2 - LVMs (40 points)

In this part, you will:
- Create an LVM that spans 4 disks
- Create a file larger than the size of any single disk to verify functionality
- Create a snapshot
- Delete the file
- Restore the LVM snapshot

In [23]:
# Install LVM
sudo apt -y install lvm2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dmeventd libaio1 libdevmapper-event1.02.1 liblvm2cmd2.03
  thin-provisioning-tools
The following NEW packages will be installed:
  dmeventd libaio1 libdevmapper-event1.02.1 liblvm2cmd2.03 lvm2
  thin-provisioning-tools
0 upgraded, 6 newly installed, 0 to remove and 3 not upgraded.
Need to get 2,448 kB of archives.
After this operation, 8,992 kB of additional disk space will be used.
Ign:1 http://deb.debian.org/debian bookworm/main amd64 libdevmapper-event1.02.1 amd64 2:1.02.185-2
Get:2 http://deb.debian.org/debian bookworm/main amd64 libaio1 amd64 0.3.113-4 [13.4 kB]
Get:3 http://deb.debian.org/debian bookworm/main amd64 liblvm2cmd2.03 amd64 2.03.16-2 [742 kB]
Get:4 http://deb.debian.org/debian bookworm/main amd64 dmeventd amd64 2:1.02.185-2 [59.2 kB]
Get:5 http://deb.debian.org/debian bookworm/main amd64 lvm2 amd64 2.03.16-2 [1,229 kB]

### Create the LVM

In [26]:
# In this cell, write a command using pvcreate to create LVM physical volumes on /dev/sdc, /dev/sdd, /dev/sde, and /dev/sdf
sudo pvcreate /dev/sdc /dev/sdd /dev/sde /dev/sdf


  Physical volume "/dev/sdc" successfully created.
  Physical volume "/dev/sdd" successfully created.
  Physical volume "/dev/sde" successfully created.
  Physical volume "/dev/sdf" successfully created.


In [27]:
# Verify that the physical volumes were created
sudo pvs

  PV         VG Fmt  Attr PSize PFree
  /dev/sdc      lvm2 ---  1.00g 1.00g
  /dev/sdd      lvm2 ---  1.00g 1.00g
  /dev/sde      lvm2 ---  1.00g 1.00g
  /dev/sdf      lvm2 ---  1.00g 1.00g


In [28]:
# In this cell, write a command using vgcreate to create a volume group named "myvg" that contains all four pvs
sudo vgcreate myvg /dev/sdc /dev/sdd /dev/sde /dev/sdf

  Volume group "myvg" successfully created


In [29]:
# Verify that the volume group was created
sudo vgs

  VG   #PV #LV #SN Attr   VSize VFree
  myvg   4   0   0 wz--n- 3.98g 3.98g


In [30]:
# In this cell, write a command using lvcreate to create a logical volume named "mylv" that occupies the entire vg
sudo lvcreate -l 100%FREE -n mylv myvg


  Logical volume "mylv" created.


In [31]:
# Verify that the logical volume was created
sudo lvs

  LV   VG   Attr       LSize Pool Origin Data%  Meta%  Move Log Cpy%Sync Convert
  mylv myvg -wi-a----- 3.98g                                                    


### Create and mount the filesystem

In [32]:
# In this cell, write the command to format /dev/myvg/mylv as ext4
sudo mkfs.ext4 /dev/myvg/mylv

mke2fs 1.47.0 (5-Feb-2023)
Discarding device blocks: done                            
Creating filesystem with 1044480 4k blocks and 261120 inodes
Filesystem UUID: a7e7ec24-d221-4264-bb75-480e7fefc9ba
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done



In [33]:
# In this cell, write the command to create the /mnt/lvm directory
sudo mkdir /mnt/lvm

In [34]:
# In this cell, write the command to mount /dev/myvg/mylvm to /mnt/lvm
sudo mount /dev/myvg/mylv /mnt/lvm

In [35]:
# Verify that it's mounted correctly
mount | grep /mnt/lvm

/dev/mapper/myvg-mylv on /mnt/lvm type ext4 (rw,relatime)


In [36]:
# Create a 2 GB file in /mnt/lvm (this may take a few seconds)
sudo dd if=/dev/zero of=/mnt/lvm/2GB_file bs=1M count=2000

2000+0 records in
2000+0 records out
2097152000 bytes (2.1 GB, 2.0 GiB) copied, 2.36902 s, 885 MB/s


In [37]:
# Verify the file was created successfully
ls -lh /mnt/lvm/2GB_file

-rw-r--r-- 1 root root 2.0G Apr 16 08:40 /mnt/lvm/2GB_file


### Re-create the LVM as RAID5

In [40]:
# In this cell, write the command to unmount /mnt/lvm using the umount command
# HINT: Make sure your terminal is not currently inside /mnt/lvm or else you will get a 'target busy' error
sudo umount /mnt/lvm

In [43]:
# Delete the lv
sudo lvremove -y /dev/myvg/mylv

  Logical volume "mylv" successfully removed.


In [44]:
# In this cell, write a command using lvcreate to create a RAID5 logical volume named "myraidlv" that occupies all four disks
sudo lvcreate --type raid5 -L 800M -n myraidlv myvg

  Using default stripesize 64.00 KiB.
  Logical volume "myraidlv" created.


In [45]:
# In this cell, write the command to format /dev/myvg/myraidlv as ext4
sudo mkfs.ext4 /dev/myvg/myraidlv

mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 204800 4k blocks and 51296 inodes
Filesystem UUID: be15dc7a-4245-430a-85ca-13a48a71df7a
Superblock backups stored on blocks: 
	32768, 98304, 163840

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (4096 blocks): done
Writing superblocks and filesystem accounting information: done



In [46]:
# In this cell, write the command to mount /dev/myvg/myraidlvm to /mnt/lvm
sudo mount /dev/myvg/myraidlv /mnt/lvm

In [47]:
# Verify that it's mounted correctly
mount | grep /mnt/lvm

/dev/mapper/myvg-myraidlv on /mnt/lvm type ext4 (rw,relatime,stripe=32)


In [48]:
# Create a 2 GB file in /mnt/lvm (this may take a few seconds)
sudo dd if=/dev/zero of=/mnt/lvm/2GB_file bs=1M count=2000

dd: error writing '/mnt/lvm/2GB_file': No space left on device
754+0 records in
753+0 records out
790433792 bytes (790 MB, 754 MiB) copied, 3.1656 s, 250 MB/s


: 1

In [49]:
# Verify the file was created successfully
ls -lh /mnt/lvm/2GB_file

-rw-r--r-- 1 root root 754M Apr 16 10:17 /mnt/lvm/2GB_file


# Short answer (10 points)

What is the capacity of the RAID5 LVM you just created? Why does it differ from the first LVM you made? Did the write speed of the 2GB file creation differ significantly? If so, why do you think that happened?

Write your answer in the cell below.

The capacity of the RAID5 LVM I created is 800 MiB which differs from the first LVM I made because it used parity across the multiple disks which reduced the usuable storage capacity. Since the first LVM did not use the RAID configuration, it had a higher capacity since it could use the full disk space. The RAID5 volume had a slower write speed (250 MB/s) compared to the non-RAID volume (800 MB/s). This is expected because RAID5 introduces overhead due to the parity calculations that must be performed, which slows down write operations.